In [10]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os
path = '/content/drive/MyDrive/FTL-Trans'
os.chdir(path)

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np

In [5]:
df_adm = pd.read_csv('DATA/ADMISSIONS.csv')
df_notes = pd.read_csv('DATA/NOTEEVENTS.csv')

df_readmission = df_adm.copy()

<ipython-input-5-b9cca7a654e3>:2: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_notes = pd.read_csv('DATA/NOTEEVENTS.csv')


In [6]:
df_readmission.ADMITTIME = pd.to_datetime(df_readmission.ADMITTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
df_readmission.DISCHTIME = pd.to_datetime(df_readmission.DISCHTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
df_readmission.DEATHTIME = pd.to_datetime(df_readmission.DEATHTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
df_readmission = df_readmission.sort_values(['SUBJECT_ID','ADMITTIME'])
df_readmission = df_readmission.reset_index(drop = True)
df_readmission['NEXT_ADMITTIME'] = df_readmission.groupby('SUBJECT_ID').ADMITTIME.shift(-1)
df_readmission['NEXT_ADMISSION_TYPE'] = df_readmission.groupby('SUBJECT_ID').ADMISSION_TYPE.shift(-1)
rows = df_readmission.NEXT_ADMISSION_TYPE == 'ELECTIVE'
df_readmission.loc[rows,'NEXT_ADMITTIME'] = pd.NaT
df_readmission.loc[rows,'NEXT_ADMISSION_TYPE'] = np.NaN
df_readmission = df_readmission.sort_values(['SUBJECT_ID','ADMITTIME'])
#When we filter out the "ELECTIVE", we need to correct the next admit time for these admissions since there might be 'emergency' next admit after "ELECTIVE"
df_readmission[['NEXT_ADMITTIME','NEXT_ADMISSION_TYPE']] = df_readmission.groupby(['SUBJECT_ID'])[['NEXT_ADMITTIME','NEXT_ADMISSION_TYPE']].fillna(method = 'bfill')
df_readmission['DAYS_NEXT_ADMIT']=  (df_readmission.NEXT_ADMITTIME - df_readmission.DISCHTIME).dt.total_seconds()/(24*60*60)
df_readmission['OUTPUT_LABEL'] = (df_readmission.DAYS_NEXT_ADMIT < 30).astype('int')
### filter out newborn and death
df_readmission = df_readmission[df_readmission['ADMISSION_TYPE']!='NEWBORN']
df_readmission = df_readmission[df_readmission.DEATHTIME.isnull()]
df_readmission['DURATION'] = (df_readmission['DISCHTIME']-df_readmission['ADMITTIME']).dt.total_seconds()/(24*60*60)

In [7]:
pos = df_readmission[df_readmission['OUTPUT_LABEL']==1]['SUBJECT_ID'].unique()
neg = df_readmission[df_readmission['OUTPUT_LABEL']==0]['SUBJECT_ID'].unique()
neg = neg[~np.isin(neg,list(pos))]
random_state = np.random.RandomState(1)
pos = np.random.choice(pos, size = 2000, replace = False)
neg = np.random.choice(neg, size = 2000, replace = False)
pos_df = df_readmission[df_readmission['SUBJECT_ID'].isin(pos)]
neg_df = df_readmission[df_readmission['SUBJECT_ID'].isin(neg)]
df_readmission = pd.concat([pos_df,neg_df])
df_readmission = df_readmission.merge(df_notes, on = ['HADM_ID'])
df_readmission = df_readmission[['HADM_ID','ROW_ID_y','CHARTDATE','CHARTTIME','TEXT','OUTPUT_LABEL']]

In [8]:
df_readmission.to_csv('DATA/readmission.csv',index=False,header=None)

In [13]:
!pip install pytorch_transformers
from pytorch_transformers import BertTokenizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 27.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-ma

In [14]:
!python3 preprocessing.py --original_data DATA/readmission.csv --output_dir DATA/raw_readmission/ --temp_dir DATA/readmission_temp/ --task_name readmission_FTL_Trans --log_path log/preprocess_readmission.txt --id_num_neg 2000 --id_num_pos 2000 --random_seed 1 --bert_model bert-base-uncased

100% 231508/231508 [00:00<00:00, 426077.53B/s]
New Pre-processing Job Start! 
original_data: DATA/readmission.csv, output_dir: DATA/raw_readmission/, temp_dir: DATA/readmission_temp/ 
task_name: readmission_FTL_Trans, log_path: log/preprocess_readmission.txt
id_num_neg: 2000, id_num_pos: 2000
random_seed: 1, bert_model: bert-base-uncased
chunk 0 tokenize start!

^C


In [ ]:
!python3 split_into_chunk.py --data_dir DATA/raw_readmission/ --train_data train.csv --val_data val.csv --test_data test.csv --log_path log/split_readmission.txt --output_dir DATA/readmission/ --max_seq_length 128

In [ ]:
!python3 run_clbert_ftlstm.py --data_dir ./DATA/readmission/ --train_data train.csv --val_data val.csv --test_data test.csv --log_path ./log_readmission.txt --bert_model ./pretraining/ --embed_mode all --task_name FTL-Trans_Prediction --max_seq_length 128 --train_batch_size 32 --eval_batch_size 1 --learning_rate 2e-5 --num_train_epochs 3 --warmup_proportion 0.1 --max_chunk_num 32 --seed 42 --gradient_accumulation_steps 1 --output_dir ./exp_FTL-Trans --save_model


in the modeling class
04/08/2024 03:07:22 - INFO - numexpr.utils -   NumExpr defaulting to 8 threads.
New Job Start! 
Data directory: ./DATA/readmission/, Directory Code: ./DATA/readmission/, Save Model: True
Output_dir: ./exp_FTL-Trans, Task Name: FTL-Trans_Prediction, embed_mode: all
max_seq_length: 128,  max_chunk_num: 32
train_batch_size: 32, eval_batch_size: 1
learning_rate: 2e-05, warmup_proportion: 0.1
num_train_epochs: 3, seed: 42, gradient_accumulation_steps: 1
FTLSTM Model's lstm_layers: 1
config setting: 
hidden_dropout_prob: 0.1 
layer_norm_eps: 1e-12 
initializer_range: 0.02 
max_note_position_embedding: 1000 
max_chunk_position_embedding: 1000 
embed_mode: all 
hidden_size: 768 
lstm_layers: 1 
task_name: FTL-Trans_Prediction 

Number of GPU is 1
Device Name: Tesla V100-SXM2-16GB,Device Capability: (7, 0)
04/08/2024 03:07:27 - INFO - pytorch_transformers.tokenization_utils -   Model name './pretraining/' not found in model shortcut name list (bert-base-uncased, bert-large